In [21]:
#LIBRAIRIE
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Dense,Concatenate
from scipy.stats import shapiro, linregress, pearsonr, spearmanr
from dcor import distance_correlation
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [9]:
# test prealable
def calculate_scores(X, Y):
    # Score de linéarité (R²)
    model = LinearRegression()
    model.fit(X.reshape(-1, 1), Y)
    S_lin = model.score(X.reshape(-1, 1), Y)
    
    # Score de normalité (Shapiro-Wilk)
    _, p_X = shapiro(X)
    _, p_Y = shapiro(Y)
    S_norm = (p_X + p_Y) / 2
    
    # Score d'outliers (IQR)
    Q1 = np.percentile(Y, 25)
    Q3 = np.percentile(Y, 75)
    IQR = Q3 - Q1
    outliers = (Y < (Q1 - 1.5 * IQR)) | (Y > (Q3 + 1.5 * IQR))
    S_out = np.sum(outliers) / len(Y)
    
    return np.array([S_lin, S_norm, S_out])


In [12]:
# Entrée : 3 scores (S_lin, S_norm, S_out)
input_scores = Input(shape=(3,))

# Couches cachées
hidden = Dense(16, activation='relu')(input_scores)
hidden = Dense(8, activation='relu')(hidden)

# Sorties : w et gamma (avec activation sigmoid pour [0, 1])
w = Dense(1, activation='sigmoid', name='w')(hidden)
gamma = Dense(1, activation='sigmoid', name='gamma')(hidden)

# Modèle
model = Model(inputs=input_scores, outputs=[w, gamma])

In [24]:
# tvaleur de tabi
def tabi_score(X, Y, w, gamma):
    # Pearson
    r, _ = pearsonr(X, Y)
    
    # Spearman
    rho, _ = spearmanr(X, Y)
    
    # Distance de corrélation
    dcor = distance_correlation(X, Y)
    
    # Facteur de complexité
    C = dcor - max(abs(r), abs(rho))
    
    # TABI
    return w * r + (1 - w) * rho + gamma * C

In [13]:
# optimisation des parametre du model 
def custom_loss(y_true, y_pred, w, gamma):
    # Calculer TABI
    tabi_value = tabi_score(y_true, y_pred, w, gamma)
    
    # Minimiser la perte
    return -tabi_value

In [15]:
#compilation du model 
model.compile(optimizer='adam', loss=custom_loss)